In [1]:
%reload_ext autoreload
%autoreload 2

In [19]:
import pandas as pd
import sys
import geopy
import geopy.distance
from geopy.distance import geodesic as GRC
import math
import utils
import pyautogui as py
import pynput
from pynput.keyboard import Controller
import time
import pyperclip
from IPython.display import clear_output
import time


## Inital data cleaning

In [20]:
data = pd.read_excel("../minesdata.xlsx")
data1 = data[["pcode", "geom", "mineral1", "workers_numb"]].copy()
data1 = data1.dropna(subset=["geom"])
data1["pcode"] = data1["pcode"].str.replace("codmine", "")
data1 = data1.astype({'pcode': 'int'})
data1["geom"] = data1["geom"].apply(lambda x: x.replace("POINT", "") if isinstance(x, str) else x)
data1["geom"] = data1["geom"].str.replace("\(|\)", "", regex=True)
data1["geom"] = data1["geom"].str.strip()
data1[["mine_long", "mine_lat"]] = data1["geom"].str.split(" ", n=1, expand=True)
data1.drop("geom", axis=1, inplace=True)

# Drop rows that contains duplicates of pcode
unique_mines = data1.drop_duplicates(subset=['pcode'])


## Find accurate location of points

In [23]:
stop = False

while not stop:    
    # check if pcode already contained for some reason
    refined_location = pd.read_excel("../refined_location.xlsx")
    
    # get new point to verify
    all_pcodes = list(map(int, unique_mines["pcode"]))
    already_checked_pcodes = list(map(int, refined_location["pcode"]))
    to_check_pcodes = list(set(all_pcodes) - set(already_checked_pcodes))
    next_pcode_to_check = to_check_pcodes[0]
    idx = all_pcodes.index(next_pcode_to_check)

    # next on list to see if far away or not
    if len(to_check_pcodes) > 1:
        next_pcode_to_check1 = to_check_pcodes[1]
    else:
        next_pcode_to_check1 = to_check_pcodes[0]
    idx1 = all_pcodes.index(next_pcode_to_check1)
    
    # longitude and latitude of point that is to check
    new_long = unique_mines.iloc[idx]['mine_long']
    new_lat = unique_mines.iloc[idx]['mine_lat']

    new_long1 = unique_mines.iloc[idx1]['mine_long']
    new_lat1 = unique_mines.iloc[idx1]['mine_lat']
    
    # Pretty print coordinates of point to check
    point = geopy.Point(latitude=new_lat, longitude=new_long)
    point1 = geopy.Point(latitude=new_lat1, longitude=new_long1)
    #clear_output(wait=True)
    print(GRC(point, point1).km)
    
    point_string = utils.pretty_print_point(point, return_var=True)
    
    # create instance of keyboard
    keyboard = Controller()
    
    # find apparent location of mine
    py.moveTo(600, 450)
    time.sleep(0.5)
    py.moveTo(80, 40)
    py.click()
    py.hotkey("command", "a")
    py.hotkey("delete")
    time.sleep(0.5)
    keyboard.type(point_string)
    py.press('enter')
    
    nr = input("Input")
    
    if nr == "0":
        additional = 0
        found = 0
        lat_refined = 0
        long_refined = 0
    elif nr == "stop0":
        additional = 0
        found = 0
        lat_refined = 0
        long_refined = 0
        stop = True 
    elif nr == "1" or nr == "stop1" or nr == "2" or nr == "stop2":
        additional = 0
        if nr == "stop1":
            stop = True
        if nr == "stop2":
            stop = True
            additional = 1
        if nr == "2":
            additional = 1
        found = 1
        py.moveTo(600, 450)
        py.click()
        py.moveTo(250, 235)
        py.click()
        py.hotkey("command", "a", interval=0.05)
        py.hotkey("command", "c", interval=0.05)
        la = pyperclip.paste()
        
        py.moveTo(250, 260)
        py.click()
        py.hotkey("command", "a", interval=0.05)
        py.hotkey("command", "c", interval=0.05)
        lo = pyperclip.paste()
    
        py.moveTo(600, 450)
        py.click()
        py.moveTo(350, 700)
        py.click()
        lat_refined, long_refined = utils.convert_dms_to_decimal(la, lo)
        print(f"Latitude: {lat_refined}, Longitude: {long_refined}")
    
    new_entry = {"pcode": next_pcode_to_check, "latitude": lat_refined, "longitude": long_refined, "found": found, "additional": additional}
    
    if not int(next_pcode_to_check) in refined_location["pcode"].values:
        refined_location = pd.concat([refined_location, pd.DataFrame([new_entry])], ignore_index=True)
    else:
        print("Pcode {} is already in the refined file...".format(next_pcode_to_check))
    
    # store new entry in file
    refined_location.to_excel("../refined_location.xlsx", index = False)

8.04605901723881
25.751037728269445
1.1911682633070968
deg:    1°39'8.11"N
parts:  1

degrees:  1
deg:   30°15'46.34"E
parts:  30

degrees:  30
Latitude: 1.65225278, Longitude: 30.26287222
7.022965758511795
deg:    1°39'30.52"N
parts:  1

degrees:  1
deg:   30°16'19.94"E
parts:  30

degrees:  30
Latitude: 1.65847778, Longitude: 30.27220556
19.206391011667748
deg:    1°42'17.94"N
parts:  1

degrees:  1
deg:   30°17'2.31"E
parts:  30

degrees:  30
Latitude: 1.70498333, Longitude: 30.283975
6.904700764933738
deg:    1°39'16.72"N
parts:  1

degrees:  1
deg:   30° 9'20.10"E
parts:  30

degrees:  30
Latitude: 1.65464444, Longitude: 30.15558333


In [18]:
stop = False
refined_location = pd.read_excel("../refined_location.xlsx")

while not stop:    
    # check if pcode already contained for some reason
    
    test_loc = pd.read_excel("../test.xlsx")
    
    # get new point to verify
    found_mines = refined_location[refined_location["found"] == 1]
    all_pcodes = list(map(int, found_mines["pcode"]))
    already_checked_pcodes = list(map(int, test_loc["pcode"]))
    to_check_pcodes = list(set(all_pcodes) - set(already_checked_pcodes))
    to_check_pcodes.sort()
    next_pcode_to_check = to_check_pcodes[0]
    print(next_pcode_to_check)
    idx = all_pcodes.index(next_pcode_to_check)

    # next on list to see if far away or not
    if len(to_check_pcodes) > 1:
        next_pcode_to_check1 = to_check_pcodes[1]
    else:
        next_pcode_to_check1 = to_check_pcodes[0]
    idx1 = all_pcodes.index(next_pcode_to_check1)
    
    # longitude and latitude of point that is to check
    new_long = unique_mines.iloc[idx]['mine_long']
    new_lat = unique_mines.iloc[idx]['mine_lat']

    new_long1 = unique_mines.iloc[idx1]['mine_long']
    new_lat1 = unique_mines.iloc[idx1]['mine_lat']
    
    # Pretty print coordinates of point to check
    point = geopy.Point(latitude=new_lat, longitude=new_long)
    point1 = geopy.Point(latitude=new_lat1, longitude=new_long1)
    #clear_output(wait=True)
    print(GRC(point, point1).km)
    
    point_string = utils.pretty_print_point(point, return_var=True)
    
    # create instance of keyboard
    keyboard = Controller()
    
    # find apparent location of mine
    py.moveTo(600, 450)
    time.sleep(0.5)
    py.moveTo(80, 40)
    py.click()
    py.hotkey("command", "a")
    py.hotkey("delete")
    time.sleep(0.5)
    keyboard.type(point_string)
    py.press('enter')
    
    nr = input("Input")
    
    if nr == "0":
        additional = 0
        found = 0
        lat_refined = 0
        long_refined = 0
    elif nr == "stop0":
        additional = 0
        found = 0
        lat_refined = 0
        long_refined = 0
        stop = True 
    elif nr == "1" or nr == "stop1" or nr == "2" or nr == "stop2":
        additional = 0
        if nr == "stop1":
            stop = True
        if nr == "stop2":
            stop = True
            additional = 1
        if nr == "2":
            additional = 1
        found = 1
        py.moveTo(600, 450)
        py.click()
        py.moveTo(250, 235)
        py.click()
        py.hotkey("command", "a", interval=0.05)
        py.hotkey("command", "c", interval=0.05)
        la = pyperclip.paste()
        
        py.moveTo(250, 260)
        py.click()
        py.hotkey("command", "a", interval=0.05)
        py.hotkey("command", "c", interval=0.05)
        lo = pyperclip.paste()
    
        py.moveTo(600, 450)
        py.click()
        py.moveTo(350, 700)
        py.click()
        lat_refined, long_refined = utils.convert_dms_to_decimal(la, lo)
        print(f"Latitude: {lat_refined}, Longitude: {long_refined}")
    
    new_entry = {"pcode": next_pcode_to_check, "latitude": lat_refined, "longitude": long_refined, "found": found, "additional": additional}
    
    if not int(next_pcode_to_check) in test_loc["pcode"].values:
        test_loc = pd.concat([test_loc, pd.DataFrame([new_entry])], ignore_index=True)
    else:
        print("Pcode {} is already in the refined file...".format(next_pcode_to_check))
    
    # store new entry in file
    test_loc.to_excel("../test.xlsx", index = False)

13
2.4793219951592365
16
62.32576741148837
17
126.13651680236809
